In [1]:
import pandas as pd
import os
import re
import numpy as np
import sys 
import config_options as cfg
chosen_day = cfg.chosen_day

In [2]:
def read_csv(dir, chosen_day):
    files = [f for f in os.listdir(dir) if chosen_day in f and f.endswith(".csv")]
    if files:
        full_path = os.path.join(dir, files[0])
        df=pd.read_csv(full_path)
        df=df.drop(df.columns[0],axis=1)
        return df
    else:
        print(f"Dia {chosen_day} no encontrado en {dir}. Revisa en la carpeta")
        return None

data_base_option = read_csv(cfg.dir_md_opciones, chosen_day)
data_base_underlying = read_csv(cfg.dir_md_subyacente, chosen_day)

In [3]:
data_base_option["biof_fecha"] = pd.to_datetime(data_base_option["biof_fecha"])
data_base_underlying["biof_fecha"] = pd.to_datetime(data_base_underlying["biof_fecha"])

def flujo_eventos(data_base_option: pd.DataFrame,
    data_base_underlying: pd.DataFrame,
    opt_cols,
    und_cols,
    time_col,
    id_col
):
    opt_cols = list(opt_cols)
    und_cols = list(und_cols)

    und = (data_base_underlying
           .sort_values(time_col)
           [[time_col] + und_cols]
           .rename(columns={c: f"{c}_under" for c in und_cols})
           .set_index(time_col))
    und_suff = [f"{c}_under" for c in und_cols]
    out = []
    for opt_id, g in data_base_option.groupby(id_col, sort=False):
        g = (g.sort_values(time_col)
               [[time_col] + opt_cols]
               .set_index(time_col))

        t = und.index.union(g.index).unique().sort_values()         # Timeline de eventos = unión de timestamps

        g_re = g.reindex(t).ffill()
        u_re = und.reindex(t).ffill()

        df = pd.concat([g_re, u_re], axis=1)

        changed_opt = df[opt_cols].ne(df[opt_cols].shift()).any(axis=1)
        changed_und = df[und_suff].ne(df[und_suff].shift()).any(axis=1)

        changed_opt = changed_opt & df[opt_cols].notna().any(axis=1)
        changed_und = changed_und & df[und_suff].notna().any(axis=1)

        if not df.empty:
            changed_opt.iloc[0] = False
            changed_und.iloc[0] = False

        block = df.reset_index().rename(columns={'index': time_col})
        block[id_col] = opt_id
        block['changed_opt'] = changed_opt.values
        block['changed_und'] = changed_und.values

        conditions = [
            changed_opt & changed_und,
            changed_opt,
            changed_und
        ]
        choices = ['BOTH','OPT','UND']
        block['event'] = np.select(conditions, choices, default='NONE')

        if not block.empty:
            block.loc[block.index[0], "event"] = "INIT"
        out.append(block)
    result = (pd.concat(out, ignore_index=True)
                .sort_values([id_col, time_col])
                .reset_index(drop=True))
    return result

data_base_merged = flujo_eventos(
     data_base_option,
     data_base_underlying,
     opt_cols=('bi_1_precio','bi_1_size','of_1_precio','of_1_size'),
     und_cols=('bi_1_precio','bi_1_size','of_1_precio','of_1_size'),
     time_col='biof_fecha',
     id_col='id_simbolo' )

In [4]:
data_base_merged["instrument"] = data_base_merged["id_simbolo"].str.extract(r"GFG([CV])")
data_base_merged["instrument"] = data_base_merged["instrument"].map({"C": "call", "V": "put"})

In [5]:
def extraer_strike(id_simbolo):
    codigo = id_simbolo.split(" - ")[2]
    match = re.search(r'(\d+)', codigo)
    if match:
        numero = int(match.group(1))
        # si termina en 3, dividir por 10
        if str(numero).endswith("3"):
            return numero / 10
        else:
            return numero
    return None
data_base_merged["strike"] = data_base_merged["id_simbolo"].apply(extraer_strike)

In [6]:
data_base_merged["intrinsic_value"] = np.where(
    data_base_merged["instrument"].str.lower() == "call",
    data_base_merged["bi_1_precio_under"] - data_base_merged["strike"],
    data_base_merged["strike"] - data_base_merged["of_1_precio_under"]
)
data_base_merged["intrinsic_value"] = data_base_merged["intrinsic_value"].clip(lower=0)

In [7]:
data_base_merged["time_value"] = np.where(
    (data_base_merged["of_1_precio"].notna()) & (data_base_merged["of_1_precio"] != 0),
    data_base_merged["of_1_precio"] - data_base_merged["intrinsic_value"],
    np.nan
)

data_base_merged = data_base_merged.dropna(subset=["time_value"])